In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 3D MNIST

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.utils import to_categorical
import matplotlib.pyplot as plt

import h5py

%matplotlib inline

In [ ]:
with h5py.File('./data/full_dataset_vectors.h5', 'r') as hf:
    x_train_ = hf["X_train"][:]
    y_train_ = hf["y_train"][:]
    x_test_ = hf["X_test"][:]
    y_test_ = hf["y_test"][:]

In [ ]:
# 1D vector to rgb values, provided by ../input/plot3d.py
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]

# Transform data from 1d to 3d rgb
def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)
n_classes = 10 # from 0 to 9, 10 labels totally

x_train = rgb_data_transform(x_train_)
x_test = rgb_data_transform(x_test_)

y_train = to_categorical(y_train_, n_classes)
y_test = to_categorical(y_test_, n_classes)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPool3D, Dense, Flatten, Dropout, BatchNormalization, Activation
from keras.optimizers import Adadelta, Adam, RMSprop

# Simple NN Model

In [ ]:
models= Sequential()
models.add(Dense(512, input_shape=(4096,)))
models.add(BatchNormalization())
models.add(Activation('elu'))
models.add(Dropout(0.25))

models.add(Dense(256))
models.add(BatchNormalization())
models.add(Activation('elu'))
models.add(Dropout(0.25))

models.add(Dense(128))
models.add(BatchNormalization())
models.add(Activation('elu'))
models.add(Dropout(0.5))

models.add(Dense(10, activation='softmax'))

In [ ]:
models.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.005), metrics=['acc'])
models.fit(x_train_, y_train_, batch_size=256, epochs=150, validation_split=0.2, verbose=2)

In [ ]:
models.evaluate(x_test_,y_test_,batch_size=256)

In modelling, test accurracy approach 1.0. Validation accuracy approch about 0.63. This model is overfitting. I found loss and validation loss is big difference and it is overfitting. I tried add dropout layer and change dropout ratio. But it is difficulty to prevent overfitting.

Loss is about 3.011 and Accuracy is about 63.9%.

# Conv3D Model

In [ ]:
model = Sequential()

model.add(Conv3D(32,(3,3,3), input_shape=(16,16,16,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(64,(3,3,3), input_shape=(16,16,16,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPool3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(32,(2,2,2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(Dropout(0.25))

model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0008), metrics=['acc'])
hist=model.fit(x_train, y_train, batch_size=128, epochs=80, validation_split=0.2, verbose=2)

In [ ]:
model.evaluate(x_test,y_test,batch_size=128)

In [ ]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']

acc = hist.history['acc']
val_acc = hist.history['val_acc']

In modelling, test accurracy approach about 0.92. Validation accuracy is almost 0.70. However, this is not accruate because of valiation loss is not stable and very fluctuated. This model has overfitting problem. I tried add dropout layer and change dropout ratio and Batchnormalization. But it is difficulty to prevent overfitting.

Loss is about 3.2 and Accuracy is about 57.8%.

In [ ]:
plt.plot(loss)
plt.plot(val_loss)
plt.xlabel('epoch')

In [ ]:
plt.plot(acc)
plt.plot(val_acc)
plt.xlabel('epoch')